In [ ]:
"""
Notebook to pose unposed cameras. Run after running 2-extrinsics.py

Pseudocode:
- Load cameras
    - If camera is posed add to posed_cameras
    - If camera is unposed add to unposed_cameras
- For each unposed camera in unposed_cameras do:
    - Find the closest posed camera
    - Select keypoints in overlapping views
    - Solve PnP to get pose of unposed camera
    - Add the posed camera to posed_cameras
    - Remove the unposed camera from unposed_cameras
    - Update extrinsics
"""





In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2

In [ ]:
# Load Cameras

In [ ]:
# 